In [1]:
#подключение библиотек и модулей
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
import re
from string import punctuation

from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymorphy3 import MorphAnalyzer

In [2]:
#функция извлечения текста из заданного URL-адреса.
def get_text_from_url(URL):
    html = urlopen(URL).read()
    soup = BeautifulSoup(html, 'html')
    return soup.get_text().replace('--','-')

In [3]:
#загрузка текстов разных авторов
shol = get_text_from_url('http://www.lib.ru/PROZA/SHOLOHOW/tihijdon34.txt')
klsh= get_text_from_url('http://www.lib.ru/HIST/KALASHNIKOW/zv_b1.txt')
chern = get_text_from_url('http://az.lib.ru/c/chernyshewskij_n_g/text_0020.shtml')

In [4]:
#деление текста на части (предложения), для которых будет устанавливаться авторство
shol_sent = nltk.sent_tokenize(shol)
klsh_sent = nltk.sent_tokenize(klsh)
chern_sent = nltk.sent_tokenize(chern)

In [5]:
#просмотр количества предложений в каждом тексте
print(len(shol_sent))
print(len(klsh_sent))
print(len(chern_sent))

22348
13629
11002


In [6]:
#создание датасетов
s1 = pd.DataFrame(data=shol_sent, columns=['text'])
k1 = pd.DataFrame(data=klsh_sent, columns=['text'])
c1 = pd.DataFrame(data=chern_sent, columns=['text'])

In [7]:
#добавление столбца с авторами
s1['author'] = ['SHOLOHOW']*len(shol_sent)
k1['author'] = ['KALASHNIKOW']*len(klsh_sent)
c1['author'] = ['CHERNYSHEWSKY']*len(chern_sent)
#объединение одинаковых частей (выбор случайных 10000 строк) трех датасетов разных авторов в один
df = pd.concat([s1.sample(10000), k1.sample(10000),c1.sample(10000)], axis=0)

In [8]:
#перемешивание
df = df.sample(30000)

In [9]:
#просмотр нескольких строк датасета
df.head()

,text,author
14229,"- Нет, все одно... Вот что я хотела сказа...",SHOLOHOW
6258,Кто должен был первый заметить это?,CHERNYSHEWSKY
9116,- с волнением спросила Вера Павловна.,CHERNYSHEWSKY
14473,"Бесцельно бродя по двору, он осмотрел скотиний...",SHOLOHOW
18086,Со степи набегал\nветерок.,SHOLOHOW


Перед обучением модели нужно подготовить датасет с предложениями: привести всё к одному регистру, очистить от знаков препинания и стоп-слов, удалить короткие предложения, произвести леммантизацию (привести слова к нормальной форме)

In [10]:
#вычисление длины строки и добавление её в новый столбец датасета
df['len_t'] = df.apply(lambda x: len(x['text']), axis=1)

In [17]:
#отсечение коротких предложений (чтобы не создавались шумы), их просмотр 
df[df.len_t<15].sample(5)

,text,author,len_t
10009,Эрмитаж).,CHERNYSHEWSKY,9
3310,"Нет, идет.",KALASHNIKOW,10
17824,То-то и оно!,SHOLOHOW,12
1926,- Тревога!..,SHOLOHOW,12
976,"О, грязь!",CHERNYSHEWSKY,9


In [18]:
#разбиение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df.text, df.author)

In [19]:
#создание экземпляра класса MorphAnalyzer
morph = MorphAnalyzer()

In [20]:
#создание списка, который содержит символы пунктуации и русские стоп-слова
stop_w = list(punctuation) + nltk.corpus.stopwords.words('russian')
#функция токенизации текста (приведения текста в нижний регистр, очистки от стоп-слов и регулярных выражений,
#и приведения в нормальную форму)
def my_tokenizer(text):
    tokens = [morph.parse(w)[0].normal_form for w in nltk.word_tokenize(text.lower())
             if (w not in stop_w)
             and not re.match('.*[.`*/0-9a-z]', w)]
    return tokens

In [21]:
#cоздание векторизатора текста
#для токенизации используется пользовательская функция my_tokenizer
#создаются униграммы (токены из одного слова)
#все признаки (токены) будут учтены без ограничений на количество
count_v = CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1,1), max_features=None)

In [22]:
#создание мешка слов
X_c = count_v.fit_transform(df.text)
#векторизация обучающей и тестовой выборок
X_train_c = count_v.transform(X_train)
X_test_c = count_v.transform(X_test)

C:\Users\tat0e\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [23]:
#просмотр длины словаря
len(count_v.vocabulary_)

22661

In [24]:
#cоздание модели логистической регрессии с параметрами C=1, max_iter=200 (для сходимости)
log_reg = LogisticRegression(C=1, max_iter=200)
#обучение модели на подготовленных данных
log_reg.fit(X_train_c, y_train)
#результат классификации
print(classification_report(y_test, log_reg.predict(X_test_c)), '\n\nCross-validation: ', 
      np.mean(cross_val_score(log_reg, X_c, df.author, cv = 4)))

               precision    recall  f1-score   support

CHERNYSHEWSKY       0.81      0.83      0.82      2440
  KALASHNIKOW       0.82      0.79      0.81      2541
     SHOLOHOW       0.78      0.79      0.78      2519

     accuracy                           0.80      7500
    macro avg       0.80      0.80      0.80      7500
 weighted avg       0.80      0.80      0.80      7500
 

Cross-validation:  0.7999999999999999


При использовании мешка слов для кодирования текста лучшая точность составила 0.7999999999999999, при отбрасывании предложений длины меннее 15 символов, кодированиии униграмм и параметрах регрессии: C=1, max_iter=200 

In [25]:
#кодироваие с помощью TF-IDF 
#для токенизации используется специальная функция токенизатора .
#рассматриваются как униграммы, так и биграммы.
#сохраняются 50 000 лучших функций (на основе оценок TF-IDF).
tfidf = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1,2), max_features=50000)
#преобразование текстовых данныч в матрицы функций TF-IDF
X_t = tfidf.fit_transform(df.text)
X_train_t = tfidf.transform(X_train)
X_test_t = tfidf.transform(X_test)

C:\Users\tat0e\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [26]:
#cоздание модели логистической регрессии с параметрами C=10, max_iter=500 (для сходимости)
log_reg = LogisticRegression(C=10, max_iter=500)
#обучение модели на подготовленных данных
log_reg.fit(X_train_t, y_train)
#результат классификации
print(classification_report(y_test, log_reg.predict(X_test_t)), '\n\nCross-validation: ', 
      np.mean(cross_val_score(log_reg, X_t, df.author, cv = 4)))

               precision    recall  f1-score   support

CHERNYSHEWSKY       0.79      0.89      0.83      2440
  KALASHNIKOW       0.83      0.79      0.81      2541
     SHOLOHOW       0.83      0.78      0.80      2519

     accuracy                           0.82      7500
    macro avg       0.82      0.82      0.82      7500
 weighted avg       0.82      0.82      0.82      7500
 

Cross-validation:  0.8128666666666666


При использовании TF-IDF для кодирования текста лучшая точность составила 0.8128666666666666, при отбрасывании предложений длины меннее 15 символов, кодированиии униграмм и биграмм, размере словаря 50000 и параметрах регрессии: C=10, max_iter=500.
Результат лучше чем при кодировании с помощью мешка слов

In [27]:
#импортируем Doc2Vec и TaggedDocument
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [28]:
#разбиение на тренировочную и тестовую выборки
train, test = train_test_split(df)

In [29]:
#создание документов с токенизированными текстами и метками авторов
train_list = train.apply(lambda x: TaggedDocument(my_tokenizer(x['text']), x['author']), axis=1)
test_list = test.apply(lambda x: TaggedDocument(my_tokenizer(x['text']), x['author']), axis=1)

In [30]:
#сздание модели Doc2Vec с параметрами vector_size=150, dm=1, window=3, epochs=50
model = Doc2Vec(vector_size=150, dm=1, window=3, epochs=50)

In [31]:
#построение словаря
model.build_vocab(train_list)

In [32]:
#обучение модели
model.train(train_list, total_examples=model.corpus_count, epochs=50)

In [33]:
#фуекция преобразования текста в векторы
def make_vector_set(model, text_list):
    X = []
    y = []
    for doc in text_list:
        X.append(model.infer_vector(doc.words))
        y.append(doc.tags)
    return np.array(X), y

In [34]:
#преобразование обучающего набора в векторы
X1_train, y1_train = make_vector_set(model, train_list)

In [35]:
#преобразование тестового набора в векторы
X1_test, y1_test = make_vector_set(model, test_list)

In [36]:
#cоздание модели логистической регрессии с параметрами C=1, max_iter=200 (для сходимости)
log_reg = LogisticRegression(C=1, max_iter=500)
#обучение модели на подготовленных данных
log_reg.fit(X1_train, y1_train)
#результат классификации
print(classification_report(y1_test, log_reg.predict(X1_test)), '\n\nCross-validation: ', 
      np.mean(cross_val_score(log_reg, X1_test, y1_test, cv = 4)))

               precision    recall  f1-score   support

CHERNYSHEWSKY       0.70      0.76      0.73      2467
  KALASHNIKOW       0.74      0.70      0.72      2536
     SHOLOHOW       0.72      0.71      0.72      2497

     accuracy                           0.72      7500
    macro avg       0.72      0.72      0.72      7500
 weighted avg       0.72      0.72      0.72      7500
 

Cross-validation:  0.7188


При использовании модели Doc2Vec для кодирования текста лучшая точность составила 0.7188, при отбрасывании предложений длины меннее 15 символов, размере вектора 150, размере окна 3 и количестве эпох обучения - 50 и параметрах регрессии: C=1, max_iter=500.
Результат получился хуже, чем при кодировании с помощью мешка слов и TF-IDF


Лучший результат показал способ кодирования TF-IDF, точность которого составила 0.8128666666666666, при отбрасывании предложений длины меннее 15 символов, кодированиии униграмм и биграмм, размере словаря 50000 и параметрах регрессии: C=10, max_iter=500.
Это можно объяснить тем, что TF-IDF учитывает не только наличие слов в документе, но и их важность в контексте всего корпуса текстов. Это позволяет выделить ключевые слова и снизить влияние часто встречающихся общеупотребительных слов.
TF-IDF может работать хорошо даже на небольших объемах данных. TF-IDF обычно менее склонен к переобучению по сравнению с более сложными моделями, такими как Doc2Vec.